In [1]:
import os 
os.chdir('/Users/egill/dev/genki/genki-signals')

In [10]:
from genki_signals.buffers import DataBuffer

buf = DataBuffer(max_size=100)

In [11]:
for i in range(1000):
    if i % 2 == 0:
        buf.append({'a': i, 'b': i})
    else:
        buf.append({'a': i})

In [12]:
buf['b']

array([800., 802., 804., 806., 808., 810., 812., 814., 816., 818., 820.,
       822., 824., 826., 828., 830., 832., 834., 836., 838., 840., 842.,
       844., 846., 848., 850., 852., 854., 856., 858., 860., 862., 864.,
       866., 868., 870., 872., 874., 876., 878., 880., 882., 884., 886.,
       888., 890., 892., 894., 896., 898., 900., 902., 904., 906., 908.,
       910., 912., 914., 916., 918., 920., 922., 924., 926., 928., 930.,
       932., 934., 936., 938., 940., 942., 944., 946., 948., 950., 952.,
       954., 956., 958., 960., 962., 964., 966., 968., 970., 972., 974.,
       976., 978., 980., 982., 984., 986., 988., 990., 992., 994., 996.,
       998.])

In [13]:
from genki_signals.data_sources import Sampler
from genki_signals.data_sources import SineWave

sine = SineWave(amplitude=3, frequency=2, phase=0)

sampler = Sampler({'sine': sine}, sample_rate=100)

In [14]:
import time
from threading import Thread

buf = DataBuffer(max_size=400)

def run_source(source, buffer):
    source.start()
    while True:
        buffer.extend(source.read())
        time.sleep(1 / 25)
        if not source.is_active:
            return
        

t = Thread(target=run_source, args=(sampler, buf))
t.start()

In [15]:
buf.plot('sine')

Figure(axes=[Axis(label='t', scale=LinearScale()), Axis(label='sine', orientation='vertical', scale=LinearScal…

In [16]:
from ipywidgets import interact, IntSlider, FloatSlider

@interact(amplitude=FloatSlider(min=1, max=5, step=0.5, value=5), frequency=IntSlider(min=1, max=7, value=3))
def set_wave_params(amplitude, frequency):
    sine.amplitude = amplitude
    sine.frequency = frequency

interactive(children=(FloatSlider(value=5.0, description='amplitude', max=5.0, min=1.0, step=0.5), IntSlider(v…

In [17]:
sampler.stop(); t.join()

In [18]:
from genki_signals.data_sources import MicDataSource, WaveDataSource, MouseDataSource, KeyboardDataSource

wave_source = WaveDataSource(ble_address='5C72E785-DAF5-1E32-599D-EBF56B8ECD5B')

buf = DataBuffer(max_size=400)

t = Thread(target=run_source, args=(wave_source, buf))
t.start()

Connecting to wave at address 5C72E785-DAF5-1E32-599D-EBF56B8ECD5B


In [20]:
from genki_signals.data_sources import RandomNoise

noise_source = RandomNoise()

source = Sampler({
    'sine_1': SineWave(amplitude=5, frequency=3, phase=0),
    'sine_2': SineWave(amplitude=3, frequency=7, phase=0.2),
    'noise': noise_source
}, sample_rate=100)

In [21]:
import genki_signals.signals as s

derived_signals = [
    s.Sum('sine_1', 'sine_2', name='composite_sine'),
    s.Sum('composite_sine', 'noise', name='wave_with_noise'),
    s.FourierTransform('wave_with_noise', window_size=256, window_overlap=64, name='wave_spectrum'),
]

In [22]:
from genki_signals.system import System

system = System(source, derived_signals)

In [23]:
buf = DataBuffer(max_size=400)

t = Thread(target=run_source, args=(system, buf))
t.start()

In [25]:
buf.plot('wave_with_noise')

Figure(axes=[Axis(label='t', scale=LinearScale()), Axis(label='wave_with_noise', orientation='vertical', scale…

In [26]:
from ipywidgets import interact, FloatSlider

@interact(noise=FloatSlider(min=0.1, max=3.0, step=0.1, value=1))
def set_noise(noise):
    noise_source.amplitude = noise

interactive(children=(FloatSlider(value=1.0, description='noise', max=3.0, min=0.1), Output()), _dom_classes=(…

In [28]:
buf.plot('wave_spectrum', plot_type='spectrogram', sample_rate=100, window_size=256)

Figure(axes=[Axis(label='Hz', scale=LinearScale()), Axis(label='db', orientation='vertical', scale=LinearScale…

In [31]:
source.stop()

In [34]:
mic_source = MicDataSource()

buf = DataBuffer(max_size=400)

sys_2 = System(mic_source, derived_signals=[])

t = Thread(target=run_source, args=(sys_2, buf))
t.start()

In [35]:
buf.keys()

dict_keys(['audio'])

In [37]:
sys_2.add_derived_signal(s.FourierTransform('audio', name='audio_spectrum', window_size=1024, window_overlap=0))

In [38]:
buf.keys()

dict_keys(['audio', 'audio_spectrum'])

In [40]:
buf.plot(key='audio_spectrum', plot_type='spectrogram', window_size=1024, sample_rate=mic_source.sample_rate)

Figure(axes=[Axis(label='Hz', scale=LinearScale()), Axis(label='db', orientation='vertical', scale=LinearScale…

In [41]:
fft = sys_2.derived_signals[0]

In [43]:
sys_2.stop()

In [46]:
mouse_source = Sampler({'mouse_position': MouseDataSource()}, sample_rate=100)

buf = DataBuffer(max_size=400)

sys_3 = System(mouse_source, derived_signals=[])

t = Thread(target=run_source, args=(sys_3, buf))
t.start()

In [49]:
buf.plot('mouse_position', plot_type='trace2D')

Figure(axes=[Axis(label='x', scale=LinearScale()), Axis(label='y', orientation='vertical', scale=LinearScale()…

In [51]:
sys_3.add_derived_signal(s.Differentiate('mouse_position', name='mouse_velocity'))

In [52]:
buf

DataBuffer(max_size=400, data=timestamp: (400,)
mouse_position: (2, 400)
mouse_velocity: (2, 400))

In [53]:
from genki_signals.models.letter_detection_model import SimpleGruModel

example_rnn_model = SimpleGruModel.load_from_checkpoint("genki_signals/models/stc_detector_final-epoch=15-val_loss=0.53.ckpt")

In [54]:
sys_3.add_derived_signal(s.Inference(example_rnn_model, input_signals=['mouse_velocity'], 
                                     stateful=True, name='model_inference'))

In [61]:
class_names=['background', 'square', 'triangle', 'circle']

buf.plot('model_inference', plot_type='histogram', class_names=class_names)

Figure(axes=[Axis(scale=OrdinalScale()), Axis(label='Probability', orientation='vertical', scale=LinearScale()…

In [62]:
sys_3.stop()